# Deploy endpoint real-time from artifacts located in S3 Buckets

In [8]:
import boto3
import sagemaker
import datetime
import pandas as pd
import awswrangler as wr

from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer

from sagemaker.model import Model
from sagemaker import get_execution_role
from sagemaker import image_uris
from sagemaker.session import Session
region = Session().boto_region_name
sagemaker_session = sagemaker.Session()
role = get_execution_role()

In [2]:
xgboost_container = sagemaker.image_uris.retrieve("xgboost", region, "1.0-1")
xgboost_container

[06/02/25 20:54:00] INFO     Defaulting to only available Python version: py3                     ]8;id=36112;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=538984;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py#610\610]8;;\

                    INFO     Defaulting to only supported image scope: cpu.                       ]8;id=132409;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=873226;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py#534\534]8;;\

'683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.0-1-cpu-py3'

In [3]:
ruta_artefacto_modelo='s3://bucket/paths.../model.tar.gz'
instancia='ml.m5.xlarge'
content_type='text/csv'
nombre_endpoint='demo-model-endpoint'

In [5]:
xgboost_model=Model(image_uri=xgboost_container,
                    model_data=ruta_artefacto_modelo,
                    role=sagemaker.get_execution_role())

In [6]:
xgboost_model.create(instance_type=instancia,
                    tags=[{'Key': 'demo', 'Value':'proyecto'}])

[06/02/25 20:56:03] INFO     Creating model with name: sagemaker-xgboost-2025-06-02-20-56-03-438    ]8;id=378609;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=562565;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#4105\4105]8;;\

In [7]:
xgboost_model.deploy(initial_instance_count=1,instance_type=instancia,
                    endpoint_name=nombre_endpoint)

[06/02/25 20:56:25] INFO     Creating model with name: sagemaker-xgboost-2025-06-02-20-56-25-612    ]8;id=258760;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=944127;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#4105\4105]8;;\

[06/02/25 20:56:26] INFO     Creating endpoint-config with name demo-model-endpoint                 ]8;id=697688;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=906768;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#6030\6030]8;;\

                    INFO     Creating endpoint with name demo-model-endpoint                        ]8;id=133375;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=589727;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#4852\4852]8;;\

-----!

In [10]:
input_data=wr.s3.read_csv("s3://bucket/.../located-file-test-csv")
input_data.head()

[06/02/25 21:01:34] INFO     Initializing a Ray instance                                               ]8;id=76366;file:///opt/conda/lib/python3.11/site-packages/awswrangler/distributed/ray/_core.py\_core.py]8;;\:]8;id=309895;file:///opt/conda/lib/python3.11/site-packages/awswrangler/distributed/ray/_core.py#202\202]8;;\

2025-06-02 21:01:36,504	WARNING services.py:2022 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 893366272 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=1.95gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2025-06-02 21:01:37,593	INFO worker.py:1786 -- Started a local Ray instance.


,0,82,0.9699098521619944,41
0,1,86,0.832443,91
1,0,74,0.212339,59
2,0,74,0.181825,79
3,0,87,0.183405,14
4,1,99,0.304242,61


In [11]:
payload =input_data.iloc[0,1:] #quitamos target
predictor = Predictor(
    endpoint_name=nombre_endpoint, sagemaker_session=sagemaker_session, serializer=CSVSerializer()
)

print(predictor.predict(payload))

b'0.9058033227920532'


/opt/conda/lib/python3.11/site-packages/sagemaker/base_serializers.py:116: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  has_multiple_rows = len(data) > 0 and self._is_sequence_like(data[0])
